In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.compat.v1 import keras

config = ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True

keras.backend.set_session(InteractiveSession(config=config))
session = InteractiveSession(config=config)

C:\Users\shinfu\anaconda3\envs\test\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\shinfu\anaconda3\envs\test\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\shinfu\anaconda3\envs\test\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\shinfu\anaconda3\envs\test\lib\site-packages\tensorflow\python\f

In [2]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator #Augmentation
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)
                     #y_train為(50000,1)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train)
        std = np.std(X_train)
        X_train = (X_train-mean)/(std+1e-7)  #1e-7為1*10的-7次方,這是為了防止當std=0時會除以0
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
one_hot = OneHotEncoder()
y_train = one_hot.fit_transform(y_train).toarray()    # y_train.shape變為(50000,10)
y_test = one_hot.transform(y_test).toarray()


classifier = Sequential()

#卷積組合
classifier.add(Conv2D(32, (3,3), input_shape=(32,32,3), activation='relu'))
classifier.add(BatchNormalization())   # BatchNormalization
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Conv2D(32, (3,3), activation='relu'))
classifier.add(BatchNormalization())  # BatchNormalization
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100, 
                     activation='relu',
                     kernel_regularizer=regularizers.l2(l=0.001))) ##regularizers
classifier.add(BatchNormalization()) ##BatchNormalization
classifier.add(Dropout(p=0.5)) ##Dropout

classifier.add(Dense(output_dim=100,
                     activation='relu',
                     kernel_regularizer=regularizers.l2(0.001)))##regularizers
classifier.add(BatchNormalization()) ##BatchNormalization
classifier.add(Dropout(p=0.3))##Dropout

classifier.add(Dense(output_dim=10,
                     activation='softmax'))
#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', 
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])


img_gen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True,
                             rotation_range=10, width_shift_range=0.1,
                             height_shift_range=0.1, shear_range=0.1, zoom_range=0.1,
                             horizontal_flip=True, vertical_flip=False, dtype=np.float32)
img_gen.fit(x_train)


earlystop = EarlyStopping(monitor='test_loss', patience=8, verbose=1) #earlystop

# 開始訓練
classifier.fit_generator(img_gen.flow(x_train, y_train, batch_size=100),
                         steps_per_epoch=500,
                         epochs=100, 
                         validation_data = (x_test, y_test),
                         callbacks = [earlystop])

(50000, 32, 32, 3)







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


C:\Users\shinfu\anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:41: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", kernel_regularizer=<keras.reg..., units=100)`
C:\Users\shinfu\anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:43: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.5)`
C:\Users\shinfu\anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:47: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", kernel_regularizer=<keras.reg..., units=100)`
C:\Users\shinfu\anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:49: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.3)`


C:\Users\shinfu\anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:52: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
500/500 [==============================] - 23s 45ms/step - loss: 2.1147 - acc: 0.3522 - val_loss: 1.6333 - val_acc: 0.4984
Epoch 2/100
 11/500 [..............................] - ETA: 5s - loss: 1.8339 - acc: 0.4045

C:\Users\shinfu\anaconda3\envs\test\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `test_loss` which is not available. Available metrics are: val_loss,val_acc,loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


500/500 [==============================] - 21s 41ms/step - loss: 1.6793 - acc: 0.4771 - val_loss: 1.4874 - val_acc: 0.5394
Epoch 3/100
500/500 [==============================] - 21s 41ms/step - loss: 1.4885 - acc: 0.5374 - val_loss: 1.2611 - val_acc: 0.6117
Epoch 4/100
500/500 [==============================] - 21s 42ms/step - loss: 1.3694 - acc: 0.5713 - val_loss: 1.2612 - val_acc: 0.6029
Epoch 5/100
500/500 [==============================] - 20s 41ms/step - loss: 1.2980 - acc: 0.5947 - val_loss: 1.2483 - val_acc: 0.6069
Epoch 6/100
500/500 [==============================] - 21s 41ms/step - loss: 1.2648 - acc: 0.6001 - val_loss: 1.1185 - val_acc: 0.6500
Epoch 7/100
500/500 [==============================] - 21s 41ms/step - loss: 1.2272 - acc: 0.6147 - val_loss: 1.1347 - val_acc: 0.6420
Epoch 8/100
500/500 [==============================] - 21s 41ms/step - loss: 1.1969 - acc: 0.6242 - val_loss: 1.0575 - val_acc: 0.6678
Epoch 9/100
500/500 [==============================] - 21s 41ms/ste

500/500 [==============================] - 21s 41ms/step - loss: 0.9811 - acc: 0.7040 - val_loss: 0.9495 - val_acc: 0.7193
Epoch 63/100
500/500 [==============================] - 20s 41ms/step - loss: 0.9858 - acc: 0.7051 - val_loss: 0.9506 - val_acc: 0.7188
Epoch 64/100
500/500 [==============================] - 21s 41ms/step - loss: 0.9865 - acc: 0.7042 - val_loss: 0.9910 - val_acc: 0.7024
Epoch 65/100
500/500 [==============================] - 20s 41ms/step - loss: 0.9850 - acc: 0.7019 - val_loss: 0.9704 - val_acc: 0.7125
Epoch 66/100
500/500 [==============================] - 20s 41ms/step - loss: 0.9807 - acc: 0.7059 - val_loss: 0.9038 - val_acc: 0.7350
Epoch 67/100
500/500 [==============================] - 20s 41ms/step - loss: 0.9803 - acc: 0.7045 - val_loss: 0.9494 - val_acc: 0.7148
Epoch 68/100
500/500 [==============================] - 20s 41ms/step - loss: 0.9823 - acc: 0.7050 - val_loss: 0.9358 - val_acc: 0.7229
Epoch 69/100
500/500 [==============================] - 21s 4